# If only the Notebook file has been downloaded from the github repos

Mount your Drive to Colab, 
Clone the github TP, 
Go in the github directory, 
Download and extract the Dataset (localy)

In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB == True:
    from google.colab import drive

    drive.mount("/content/gdrive")

    %cd /content/gdrive/MyDrive

    # if wildfire_segmentation folder doesn't exist then clone the github project in /content/gdrive/MyDrive
    ! [ ! -d Wildfire_segmentation ] && git clone "https://github.com/ThomasLOUIS1/Wildfire_segmentation.git"

    # Go to the github project folder
    %cd Wildfire_segmentation/data

    # if dataset.zip dosn't exist then download it
    ! [ ! -f dataset.zip ] && echo "dataset.zip dosn't exist" && wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1idUH24mGyAegB0YzDcJmd9XyNVbqGsnS' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1idUH24mGyAegB0YzDcJmd9XyNVbqGsnS" -O dataset.zip && rm -rf /tmp/cookies.txt

    # if sample_data/RGB dosn't exist then exctract the dataset into sample_data/
    ! [ ! -d /content/sample_data/RGB ] && echo "data dosn't exist" && unzip -qq dataset.zip -d /content/sample_data

    data_dir = "/content/sample_data"

    %cd /content/gdrive/MyDrive/Wildfire_segmentation
else : 
    # if dataset.zip dosn't exist then download it
    ! [ ! -f data/dataset.zip ] && echo "dataset.zip doesn't exist" && wget -P data/ --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1idUH24mGyAegB0YzDcJmd9XyNVbqGsnS' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1idUH24mGyAegB0YzDcJmd9XyNVbqGsnS" -O dataset.zip && rm -rf /tmp/cookies.txt
    
    # if data/RGB dosn't exist then exctract the dataset into data/
    ! [ ! -d data/RGB ] && echo "data doesn't exist" && unzip -qq data/dataset.zip -d data/

    data_dir = "data/"

    import os
    os.environ['CUDA_VISIBLE_DEVICES'] = "0"

# Wildfire Segmentation with Multi-spectral Images

### What are multi-spectrale Images and why using it for Wildfire Segmentation

Multi-spectral images can be used for wildfire segmentation because they capture information about the target in multiple wavelength bands, which can provide more information about the characteristics of the wildfire and the surrounding area. 

For example, near-infrared (NIR) bands can be used to identify vegetation, while shortwave infrared (SWIR) bands can be used to detect the presence of smoke. Additionally, using multiple wavelength bands can help to reduce the impact of atmospheric conditions, such as clouds and haze, on the image. This can improve the accuracy of the segmentation and make it easier to identify the wildfire in the image.

These images are stored in .tif files. .tif (TIFF) is a widely-used file format for images. It is capable of storing images in a lossless format, meaning that no data is lost when the image is compressed. This makes it a popular choice for storing high-quality images, such as those used in professional photography, printing or satellite imagery. 

# 0.1 Get dataset files paths

Loading a segmentation dataset differ from loading a classification dataset with a tensorflow or Keras function.

You don't load images from classes, you load images not bellonging to any classes and masks/targets that contains one or more classes.

In [ ]:
# Imports
import os
import PIL
from PIL import Image
import tensorflow as tf


######################################
# Complete the folowing code replacing "______" : 
######################################
# Dataset folder paths declaration
RGB_dir = data_dir + "/______"                    # Only for display purpose
triband_dir = data_dir + "/______"    # Data directory
target_dir = data_dir + "/______"          # Labels directory

# Define images size
img_size = (______, ______)

######################################
######################################

######################################
# What's the purpose of the following function ?
# Answer : ___________________________________.
def sort_tif_paths_from_folder(dir):
    """
    Get all tif files sorted by name in a directory and return as a sorted list.
    
    Parameters:
    dir (str): directory path containing the tif files

    Returns:
    list: sorted list of tif file paths in the directory
    """
    paths_list = sorted(
    [
        os.path.join(dir, fname)
        for fname in os.listdir(dir)
        if fname.endswith(".tif")
    ]
    )
    return paths_list
######################################
######################################

# Get sorted list of tif files for RGB images
RGB_img_paths = sort_tif_paths_from_folder(RGB_dir)

######################################
# Complete the folowing code replacing "______" : 
######################################
# Get sorted list of tif files for triband training images
triband_img_paths_train = sort_tif_paths_from_folder(triband_dir + "/______" )
# Get sorted list of tif files for triband validation images
triband_img_paths_val = sort_tif_paths_from_folder(triband_dir + "/______")
# Get sorted list of tif files for triband test images
triband_img_paths_test = sort_tif_paths_from_folder(triband_dir + "/______")

# Get sorted list of tif files for target training images
mask_img_paths_train = sort_tif_paths_from_folder(target_dir + "/______")
# Get sorted list of tif files for target validation images
mask_img_paths_val = sort_tif_paths_from_folder(target_dir + "/______")
# Get sorted list of tif files for target test images
mask_img_paths_test = sort_tif_paths_from_folder(target_dir + "/______")
######################################
######################################

######################################
# Complete the folowing code replacing "______" : 
# The idea is to compute the number of samples we have. 
# Tips : you have to sum the length of triband_img_paths_train, triband_img_paths_val and triband_img_paths_test arrays
######################################
print("Number of samples from {} : {}".format(triband_dir, ______))
print("Number of samples from {} : {}".format(target_dir, ______)
######################################
######################################

# Print 6 firsts paths from RGB, biband and target paths
# Note: Only works for the first 15 because we don't have many RGB images
for RGB_path, triband_path, target_path in zip(RGB_img_paths[:6], triband_img_paths_train[:6], mask_img_paths_train[:6]):
    print(RGB_path, "|", triband_path, "|", target_path)


# 0.2 Visualize RGB / Tri-bands / mask datas

First we import a function to display several images (declared in /utils.py) and others utils

In [ ]:
# Display 1 or more numpy matrix 
from utils import display_matrix

# Used to load/Read/ and plot images 
import matplotlib.pyplot as plt

# Used to perform manipulation on matrix/images
import numpy as np


### Dataset Visualization 

We visualize the RBG image, the tri-bands image and the true ground mask.

You will see that with the RGB image, it's very difficult to see the fire... But with the tri-bands image, it's quite easy !

In [ ]:
i = 0
n_images_to_display = 3
for id in range(len(RGB_img_paths)):
    # Read the mask data for the current iteration
    image_masks = Image.open(mask_img_paths_train[id])
    # Check if there's at least one "Fire" pixel and if the number of images displayed is less than 3

    if np.max(image_masks) > 0 and i < n_images_to_display:
        # Read RGB and Tri-bands data
        image_RGB = Image.open(RGB_img_paths[id])
        image_triband = Image.open(triband_img_paths_train[id])

        # print id and path of the associated RGB path
        print("Image n {}, path : {}".format(id, RGB_img_paths[id]))
        
        # Store data in matrix and display it
        matrix = [[image_RGB, image_triband, image_masks]]
        display_matrix(matrix, title_list=["RGB", "Tri-Bands", "Ground Truth Mask"])
        
        # Increment counter for number of images displayed
        i += 1

    # Break out of loop if 3 images have been displayed
    if i == n_images_to_display:
        break

### <span style="color:red">What do you think of Tribands images compared to RGB images ?</span>

**Put your answer bellow** : 

# 0.3 Load dataset

We will now load the dataset to use it later for training.

First we define a function to create a dataset from paths gathered previously.

In [ ]:
batch_size = 16
######################################
# Complete the folowing code replacing "______" : 
######################################
def load_triband_and_target_from_paths(paths = None):
    """
    Load the tri-bands and target images data from the given paths and convert them into a tensorflow dataset.
    
    Parameters:
    paths (List): List of tuples of tri-bands and target image paths
    
    Returns:
    data (tf.data.Dataset): A tensorflow dataset object with tri-bands and target images data.
    """
    
    # Initialize arrays to store tri-bands image data and target image data
    x = np.zeros((len(paths),) + img_size + (3,), dtype="float32")
    y = np.zeros((len(paths),) + img_size + (1,), dtype="float32")
    
    # Loop through the list of tri-bands and target image paths
    for i, (triband_path, mask_path)  in enumerate(paths):
        
        # Read the tri-bands image file and normalize the data
        triband = np.array(Image.open(______)) / ______
        x[i] = triband
        
        # Open the target image file
        mask = np.array(Image.open(______))
        # Add an extra dimension to the target data for compatibility with the model
        mask = np.expand_dims(mask, 2)
        y[i] = mask
        
    # Create a tensorflow dataset from tri-bands and target image data
    data = tf.data.Dataset.from_tensor_slices((x, y))
    
    # Batch the dataset and fetch the data in advance for faster processing
    data = data.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    return data
######################################
######################################

And we use it to load train, val and test sets

In [ ]:
# Create tuples of (Tri-bands image path, Target image path) for train, val and test datasets
train_paths = list(zip(triband_img_paths_train, mask_img_paths_train))
val_paths = list(zip(triband_img_paths_val, mask_img_paths_val))
test_paths = list(zip(triband_img_paths_test, mask_img_paths_test))


######################################
# Complete the folowing code replacing "______" : 
######################################
# Load train, val and test datasets from the tuple of Tri-bands and Target image paths
train_ds =  load_triband_and_target_from_paths(______)
val_ds = load_triband_and_target_from_paths(______)
test_ds = load_triband_and_target_from_paths(______)
######################################
######################################

# 1.0 Build the model

Here we will create a very classical Unet-like CNN model that we'll be used for each training.

### <span style="color:red">Create the Unet-like CNN model with the following architecture : </span>
1. A 2D convolution with 8 filters of 3*3 and a ReLU activation 
2. A Max Pooling 2D with 2*2 kernel 
  ---
3. A 2D convolution with 4 filters of 3*3 and a ReLU activation 
  ---
4. A Dropout layer at 25%
  ---
5. A 2D Transpose Convolution with 4 filters of 3*3 and a ReLU activation 
6. A Up Sampling 2D with 2*2 kernel 
  ---
7. A 2D Transpose Convolution with 8 filters of 3*3 and a ReLU activation 
8. A 2D Convolution with 1 filter of 3*3, a ReLU activation with a same padding

At the end display the model.

In [ ]:
from keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, Dropout, UpSampling2D
from tensorflow.keras.optimizers import Adam

In [ ]:
######################################
# Complete the code with the architecture of the model described above : 
######################################

model = Sequential()

model.add(___________)

...

model.add(___________)

######################################
######################################

model.summary()

# 1.1 Clone the 1st model

We will copy the model into a new "model1". Doing that we allow us to start with the same model for each training

In [ ]:
from tensorflow.keras.models import clone_model

model1 = clone_model(model)

model1.summary()

### <span style="color:red"> Compile the model with the following parameters : </span>
 - An Adam optimizer with a learning rate at 0.01
 - A Binary Accuracy metrics
 - A Binary Crossentropy loss

Add Optimizer, Metric(s), Loss(es) and compile the model1

In [ ]:
from tensorflow.keras.optimizers import Adam

######################################
# Complete the folowing code replacing "______" : 
######################################

model1.compile(optimizer = ______, metrics = ______, loss = ______)

######################################
######################################

# 1.2 Train the model1

Now we will train the model1 for 4 epochs with "train_ds" dataset for trainong and "val_ds" for validation.

In [ ]:
######################################
# Complete the folowing code replacing "______" : 
######################################

history = model1.fit(______, epochs = ______, validation_data = ______)

######################################
######################################

Evaluate model1 with the test_ds dataset >>

In [ ]:
# Import a function that display scores and values (see utils.py)
from utils import print_score

######################################
# Complete the folowing code replacing "______" : 
######################################

model1_score = ______.evaluate(______)

######################################
######################################

# Display model1 scores
print_score(model1_score)

### <span style="color:red">What do you think of the model1 scores ?</span>

**Put your answer bellow** : 

# Is this score really accurate ?

Let's check this with diplaying some prediction from the test set >>

In [ ]:
# Import a predict function similar to model.predict but with a treshhold at 0.5 (see utils.py)
from utils import predict

# A function that display firsts n_range tri-bands, masks and predictions
def display_sample_prediction(model, triband_paths, mask_paths, n_range = 3): 
  i = 0
  n_range = n_range

  # Pass through len(RGB_img_paths) firsts images paths
  for id in range(len(RGB_img_paths)):
    # Read the mask data for the current iteration
    image_masks = Image.open(mask_paths[id])
    # Check if there's at least one "Fire" pixel and if the number of images displayed is less than n_range

    if np.max(image_masks) > 0 and i < n_range:
          
      n_triband = np.array(Image.open(triband_paths[id]))/255.0
      tribands = np.expand_dims(n_triband, 0)

      # print id and path of the associated Tri-bands path
      print("Image n {}, path : {}".format(id, triband_paths[id]))
      
      predicted_mask = predict(model, tribands)
      predicted_mask = np.squeeze(predicted_mask)

      matrix=[[n_triband, image_masks, predicted_mask]] # [[Tribands,mask,predict]]

      display_matrix(matrix ,title_list=['Tri-bands', 'GT Mask', 'Predicted'])
      
      # Increment counter for number of images displayed
      i += 1
    
    # Break out of loop if n_range images have been displayed
    if i == n_range:
        break

######################################
# Complete the folowing code replacing "______" : 
######################################

# Diplay Tri-bands, mask and prediction of the model1
display_sample_prediction(______, ______, ______)

######################################
######################################

### <span style="color:red">What do you think of the Predictions compared to the Masks ?</span>

**Put your answer bellow** : 

### The effect of an unbalanced dataset

The problem comes from the fact that there are more "No_fire" pixels than "Fire" pixels, much much more. 

Then the neural network don't find interesting to give importance to these pixels, then it will classify all pixels as "No_fire"... 

We can see it with a Confusion Matrix >>

In [ ]:
from utils import display_confusion_matrix, load_masks

test_gt_masks = load_masks(mask_img_paths_test, img_size=img_size)

display_confusion_matrix(predict(model1, test_ds), test_gt_masks)

# 1.3 Dataset balancing

There are several ways to prevent bad generalization from an unbalanced dataset such as :

 - Resampling techniques such as oversampling the minority class or undersampling the majority class to balance the dataset.

 - Using a threshold in the model's decision function to adjust the trade-off between precision and recall.

 - Collecting more data to balance the classes or using a different dataset.

But here the simpliest way for us is to use a weighted loss function to give more importance to the minority class during training.

On the top of that we will use more metrics allowing us to see if the "Fire" class is properly classify.

# 2.1 Clone and train the 2nd model

Here we will clone the same Unet-like CNN model but we will balance the dataset during training.

To do so we will use a loss giving more "weight" to the "Fire" class as it is really rare.

In [ ]:

model2 = clone_model(model)

model2.summary()

# 2.2 Train with weighted loss

This time we will use the weighted binary crossentropy loss defined bellow.

But we need to choose the weight for each classes "Fire" and "No_fire", to do so we have 3 choices :

 - Try iteratively several weights ( Could be very long )
 - Use backprop to compute these parameters ( Could be difficult )
 - Compute the number of "Fire" pixels and "No_Fire" pixels to get a proportion of each pixels

 Here we will try to compute the number of "Fire" pixels and "No_Fire" pixels in our train set.

In [ ]:
zeros = 0
ones  = 0
for x, y in train_ds:
  zeros += np.count_nonzero(y == 0)
  ones += np.count_nonzero(y == 1)

ones_proportion = ones/(min(ones,zeros))
zeros_proportion = zeros/(min(ones,zeros))

ones_weight = zeros_proportion
zeros_weight = ones_proportion

print("Number of ones : {} | Number of zeros : {}".format(ones, zeros))
print("Ones proportion : {} | Zeros proportion : {}".format(ones_proportion, zeros_proportion))
print("Ones weight : {} | Zeros weight : {}".format(ones_weight, zeros_weight))

These results means that for 1 "Fire" pixel, we have around 550 "No_Fire" pixels. Meaning our dataset is clearly unbalanced.

We could use the proportion of "No_Fire" pixel (550) as a class weight for "Fire" class but this is a too big number to use it weighted binary crossentropy loss, in fact if we try this our model will not converge... One reason is if in one batch there is a too large amount of "Fire" pixels, the loss will "explode" and diverge. One way to prevent this could be too reduce the learning rate. 

In the case of strong unbalanced classes, the iterative way of founding the best class weight is mandatory.

We will set this class weight to 26 as we tested some for the TP. In another notebook you will be able to try different class weights.

In [ ]:
ones_weight = 26
zeros_weight = 1.0

### Create a custom loss function allowing to pu weight for each class :
To do so, the idea is to declare a function **weighted_binary_crossentropy** that takes two inputs, **y_true** and **y_pred**, which represent the true label and the predicted label respectively. The **y_true** and **y_pred** are clipped to a small value close to 0 (K.epsilon()) to avoid numerical instability in the computation of logarithms. 

The loss is computed as the weighted negative log-likelihood of the true label, where the weight is determined by two variables ones_weight and zeros_weight. The loss is then averaged across all samples (axis=-1) and returned as the final result.

In [ ]:
from tensorflow.keras import backend as K

def weighted_binary_crossentropy( y_true, y_pred):
    # Clipping y_true and y_pred to avoid numeric instability
    y_true = K.clip(y_true, K.epsilon(), 1-K.epsilon())
    y_pred = K.clip(y_pred, K.epsilon(), 1-K.epsilon())

    # Calculating the weighted negative log-likelihood loss
    # The original logaritic loss is : loss = -(y_true * K.log(y_pred) + (1 - y_true) * K.log(1 - y_pred))
    loss = -(y_true * K.log(y_pred) * ones_weight + (1 - y_true) * K.log(1 - y_pred) * zeros_weight)

    # Averaging the loss across all samples
    return K.mean(loss, axis=-1)

# 2.3 Adding more metrics

For this training we need more than the Binary Accuracy metrics as it is not enough to see if the model converge.

We will use 3 metrics :
 - Recall 
 - Precision 
 - F1-score 


### <span style="color:red"> Give an explanation for each metrics :</span>

**Put your answer bellow** : 

 - Recall is ...

 - Precision is ...

 - F1-score is ...

### <span style="color:red"> Compile model2 with the following parameters : </span>
 - An Adam optimizer with a learning rate at 0.01
 - The weighted binary crossentropy loss created above
 - Several metrics :
   - Binary Accuracy
   - Recall
   - Precision
   - F1-score

In [ ]:
from metrics_and_losses import recall_m, precision_m, f1_m

######################################
# Complete the folowing code replacing "______" : 
######################################

metrics = [______, ______, ______, ______]

______.compile(optimizer = ______, metrics = ______, loss = ______)

######################################
######################################

# 2.4 Train model2

Now we will train the model2 for 4 epochs with "train_ds" dataset for trainong and "val_ds" for validation.

In [ ]:
######################################
# Complete the folowing code replacing "______" : 
######################################

history = model2.fit(______, epochs = ______, validation_data = ______)

######################################
######################################

Evaluate model2 with the test_ds dataset >>

In [ ]:
######################################
# Complete the folowing code replacing "______" : 
######################################

model2_score = ______.evaluate(______)

######################################
######################################

# Display model1 scores
print_score(model2_score)

## Better results ?

Again we will see if the model works better.

In [ ]:

######################################
# Complete the folowing code replacing "______" : 
######################################

# Diplay Tri-bands, mask and prediction of the model2
display_sample_prediction(______, ______, ______)

######################################
######################################

### <span style="color:red">What do you think of the Predictions compared to the Masks ?</span>

**Put your answer bellow** : 

Now we can check again the Confusion Matrix >>

In [ ]:
from utils import display_confusion_matrix, load_masks

# test_gt_masks = load_masks(mask_img_paths_test) # already load

display_confusion_matrix(predict(model2, test_ds), test_gt_masks)

That's better, but can we improve the F1 score ?

# 3.1 clone and train the 3rd model

Again we build the model

In [ ]:
model3 = clone_model(model)

model3.summary()

# 3.2 Train with Dice/F1-score loss
The Dice loss is a loss function commonly used in image segmentation tasks. It is a variation of the Sørensen-Dice coefficient, which is a measure of similarity between two sets. The Dice loss calculates the difference between the predicted segmentation and the ground truth segmentation, with a range of 0 to 1, where 0 indicates no overlap and 1 indicates a perfect overlap. The Dice loss is defined as 1 - (2 * (intersection of predicted and ground truth) / (size of predicted + size of ground truth)). Lower values of the Dice loss indicate a better match between the predicted and ground truth segmentations.

Bellow you will found an implementation of it :

In [ ]:
# The Dice loss function we will use in the model.compile
def dice_loss(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return 1 - (2. * intersection + 1.) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1.)


Then compile it...

### <span style="color:red"> Compile model3 with the following parameters : </span>
 - An Adam optimizer with a learning rate at 0.01
 - The Dice loss created above
 - Several metrics :
   - Binary Accuracy
   - Recall
   - Precision
   - F1-score

In [ ]:
# from metrics_and_losses import recall_m, precision_m, f1_m

######################################
# Complete the folowing code replacing "______" : 
######################################
metrics = [______, ______, ______, ______]

______.compile(optimizer = ______, metrics = ______, loss = ______)

######################################
######################################

Now we will train the model3 for 4 epochs with "train_ds" dataset for trainong and "val_ds" for validation.

In [ ]:
######################################
# Complete the folowing code replacing "______" : 
######################################

history = model3.fit(______, epochs = ______, validation_data = ______)

######################################
######################################

### We will see that the f1_score is better with the Dice loss

Evaluate model3 with the test_ds dataset >>

In [ ]:
######################################
# Complete the folowing code replacing "______" : 
######################################

model3_score = ______.evaluate(______)

######################################
######################################

# Display model1 scores
print_score(model3_score)

In [ ]:

######################################
# Complete the folowing code replacing "______" : 
######################################

# Diplay Tri-bands, mask and prediction of the model3
display_sample_prediction(______, ______, ______)

######################################
######################################

### <span style="color:red">What do you think of the Predictions compared to the Masks ?</span>

**Put your answer bellow** : 

Now we can check again the Confusion Matrix >>

In [ ]:
from utils import display_confusion_matrix, load_masks

# test_gt_masks = load_masks(mask_img_paths_test) # already load

display_confusion_matrix(predict(model3, test_ds), test_gt_masks)

Now let's try a combination of weighted loss and dice loss.

# 4.1 Create and train the 4th model

In [ ]:
model4 = clone_model(model)

model4.summary()

We define a loss which combine a weighted binary crossentropy loss and a dice loss

In [ ]:
def weighted_binary_crossentropy_and_dice(y_true, y_pred):
    return weighted_binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)

### <span style="color:red"> Compile model4 with the following parameters : </span>
 - An Adam optimizer with a learning rate at 0.01
 - The loss combination of weighted binary crossentropy and dice loss created above
 - Several metrics :
   - Binary Accuracy
   - Recall
   - Precision
   - F1-score

In [ ]:
######################################
# Complete the folowing code replacing "______" : 
######################################

metrics = [______, ______, ______, ______]

______.compile(optimizer = ______, metrics = ______, loss = ______)

######################################
######################################

Now we will train the model4 for 4 epochs with "train_ds" dataset for trainong and "val_ds" for validation.

In [ ]:
######################################
# Complete the folowing code replacing "______" : 
######################################

history = model4.fit(______, epochs = ______, validation_data = ______)

######################################
######################################

Evaluate model4 with the test_ds dataset >>

In [ ]:
######################################
# Complete the folowing code replacing "______" : 
######################################

model4_score = ______.evaluate(______)

######################################
######################################

# Display model1 scores
print_score(model4_score)

And display some prediction.

In [ ]:
######################################
# Complete the folowing code replacing "______" : 
######################################

# Diplay Tri-bands, mask and prediction of the model4
display_sample_prediction(______, ______, ______)

######################################
######################################

### <span style="color:red">What do you think of the Predictions compared to the Masks ?</span>

**Put your answer bellow** : 

Now we can check again the Confusion Matrix >>

In [ ]:
from utils import display_confusion_matrix, load_masks

# test_gt_masks = load_masks(mask_img_paths_test) # already load

display_confusion_matrix(predict(model4, test_ds), test_gt_masks)

# 5.1 Compare all runs !

Write a code to compare each scores of each runs using print_score(modelX_score) :

Don't forget to re-evaluate model1 with recall_m, precision_m, f1_m metrics.

In [ ]:
######################################
# Complete the folowing code replacing "______" : 
######################################

metrics = [______, ______, ______, ______]

______.compile(metrics = ______)
model1_score = ______.evaluate(______)


print("\n model1_score :")
print_score(model1_score)
print("\n model2 weighted loss :")
print_score(model2_score)
print("\n model3 Dice_loss :")
print_score(model3_score)
print("\n model4 weighted loss + Dice_loss :")
print_score(model4_score)

######################################
######################################


#6. Bonus

Go to the Bonus_wildfire.ipynb Notebook.
If you have time left, try finding the best combination of loss/weight/epochs/architecture to get a test f1_score higher than 98%.